#   Outdoor Wheather

In [19]:
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd


In [20]:

!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas


import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [21]:


def make_dataframe(openmeteo_response):

    hourly = openmeteo_response.Hourly()
    _start = pd.to_datetime(hourly.Time(), unit = "s", utc = True)
    _end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True)
    _delta = pd.Timedelta(seconds = hourly.Interval())

    data = {
        "date": pd.date_range(_start, _end, freq=_delta, inclusive="left"),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "surface_pressure": hourly.Variables(1).ValuesAsNumpy(),
        "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
        "wind_direction_10m": hourly.Variables(3).ValuesAsNumpy(),
        "wind_gusts_10m": hourly.Variables(4).ValuesAsNumpy(),
    }

    return pd.DataFrame(data).set_index("date")

In [22]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [47.36, 46, 47.26, 46.5],
	"longitude": [8.55, 8.95, 11.39, 11.35],
	"start_date": "2010-01-01",
	"end_date": "2024-03-20",
	"hourly": ["temperature_2m", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"]
}
data = openmeteo.weather_api(url, params=params)
#data.to_csv('data_fohn.csv')
data = dict(zip(("Zurich", "Lugano", "Innsbruck", "Bolzano"), data))
data = pd.concat({key: make_dataframe(val) for key, val in data.items()})

city_codes = {"Zurich": 1, "Lugano": 2, "Innsbruck": 3, "Bolzano": 4}
data['city_code'] = data.index.get_level_values(0).map(city_codes)

In [23]:
data


temperature_2m  surface_pressure  \
        date                                                          
Zurich  2010-01-01 00:00:00+00:00           0.894        942.275085   
        2010-01-01 01:00:00+00:00           0.494        941.824402   
        2010-01-01 02:00:00+00:00          -0.006        941.640625   
        2010-01-01 03:00:00+00:00          -0.306        941.112305   
        2010-01-01 04:00:00+00:00          -0.556        940.593262   
...                                           ...               ...   
Bolzano 2024-03-20 19:00:00+00:00             NaN               NaN   
        2024-03-20 20:00:00+00:00             NaN               NaN   
        2024-03-20 21:00:00+00:00             NaN               NaN   
        2024-03-20 22:00:00+00:00             NaN               NaN   
        2024-03-20 23:00:00+00:00             NaN               NaN   

                                   wind_speed_10m  wind_direction_10m  \
        date                                                            
Zurich  2010-01-01 00:00:00+00:00        1.138420          341.564941   
        2010-01-01 01:00:00+00:00        1.018234           44.999897   
        2010-01-01 02:00:00+00:00        1.297998          123.690094   
        2010-01-01 03:00:00+00:00        1.800000          143.130020   
        2010-01-01 04:00:00+00:00        1.138420           71.564964   
...                                           ...                 ...   
Bolzano 2024-03-20 19:00:00+00:00             NaN                 NaN   
        2024-03-20 20:00:00+00:00             NaN                 NaN   
        2024-03-20 21:00:00+00:00             NaN                 NaN   
        2024-03-20 22:00:00+00:00             NaN                 NaN   
        2024-03-20 23:00:00+00:00             NaN                 NaN   

                                   wind_gusts_10m  city_code  
        date                                                  
Zurich  2010-01-01 00:00:00+00:00            9.00          1  
        2010-01-01 01:00:00+00:00            9.00          1  
        2010-01-01 02:00:00+00:00            9.36          1  
        2010-01-01 03:00:00+00:00            9.36          1  
        2010-01-01 04:00:00+00:00           10.08          1  
...                                           ...        ...  
Bolzano 2024-03-20 19:00:00+00:00             NaN          4  
        2024-03-20 20:00:00+00:00             NaN          4  
        2024-03-20 21:00:00+00:00             NaN          4  
        2024-03-20 22:00:00+00:00             NaN          4  
        2024-03-20 23:00:00+00:00             NaN          4  

[498528 rows x 6 columns]

In [24]:
data.to_csv('data_fohn.csv')

In [25]:
data.reset_index(inplace=True)
data['Time'] = data['date'].dt.time
data['Date'] = data['date'].dt.date
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498528 entries, 0 to 498527
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   level_0             498528 non-null  object             
 1   date                498528 non-null  datetime64[ns, UTC]
 2   temperature_2m      498432 non-null  float32            
 3   surface_pressure    498432 non-null  float32            
 4   wind_speed_10m      498432 non-null  float32            
 5   wind_direction_10m  498432 non-null  float32            
 6   wind_gusts_10m      498432 non-null  float32            
 7   city_code           498528 non-null  int64              
 8   Time                498528 non-null  object             
 9   Date                498528 non-null  object             
dtypes: datetime64[ns, UTC](1), float32(5), int64(1), object(3)
memory usage: 28.5+ MB


In [26]:
data

,level_0,date,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code,Time,Date
0,Zurich,2010-01-01 00:00:00+00:00,0.894,942.275085,1.138420,341.564941,9.00,1,00:00:00,2010-01-01
1,Zurich,2010-01-01 01:00:00+00:00,0.494,941.824402,1.018234,44.999897,9.00,1,01:00:00,2010-01-01
2,Zurich,2010-01-01 02:00:00+00:00,-0.006,941.640625,1.297998,123.690094,9.36,1,02:00:00,2010-01-01
3,Zurich,2010-01-01 03:00:00+00:00,-0.306,941.112305,1.800000,143.130020,9.36,1,03:00:00,2010-01-01
4,Zurich,2010-01-01 04:00:00+00:00,-0.556,940.593262,1.138420,71.564964,10.08,1,04:00:00,2010-01-01
...,...,...,...,...,...,...,...,...,...,...
498523,Bolzano,2024-03-20 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,4,19:00:00,2024-03-20
498524,Bolzano,2024-03-20 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,4,20:00:00,2024-03-20
498525,Bolzano,2024-03-20 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN,4,21:00:00,2024-03-20
498526,Bolzano,2024-03-20 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,4,22:00:00,2024-03-20


In [27]:
data.value_counts()

level_0    date                       temperature_2m  surface_pressure  wind_speed_10m  wind_direction_10m  wind_gusts_10m  city_code  Time      Date      
Bolzano    2010-01-01 00:00:00+00:00  4.138000        959.277893        1.080000        90.000000           6.120000        4          00:00:00  2010-01-01    1
Lugano     2019-06-23 12:00:00+00:00  25.933001       984.360291        7.928177        230.527573          23.759998       2          12:00:00  2019-06-23    1
           2019-06-24 16:00:00+00:00  28.833000       987.948120        5.588703        255.068527          18.000000       2          16:00:00  2019-06-24    1
           2019-06-24 15:00:00+00:00  29.132999       987.589905        5.804825        240.255203          20.160000       2          15:00:00  2019-06-24    1
           2019-06-24 14:00:00+00:00  29.183001       987.691956        6.608722        240.642334          20.160000       2          14:00:00  2019-06-24    1
                                       

In [28]:
data.describe()

,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code
count,498432.000000,498432.000000,498432.000000,498432.000000,498432.000000,498528.000000
mean,11.732435,971.064392,4.911405,175.865158,19.896273,2.500000
std,8.334503,16.162291,3.250290,115.199409,11.019178,1.118035
min,-21.107498,907.231873,0.000000,0.924034,1.440000,1.000000
25%,5.291500,957.620178,2.811690,60.642342,12.240000,1.750000
50%,11.583000,974.013977,4.334974,180.000000,16.919998,2.500000
75%,18.033001,984.231140,6.130579,276.340088,24.840000,3.250000
max,39.786503,1012.088501,40.938820,360.000000,132.839996,4.000000


In [29]:
data.isnull()

,level_0,date,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code,Time,Date
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
498523,False,False,True,True,True,True,True,False,False,False
498524,False,False,True,True,True,True,True,False,False,False
498525,False,False,True,True,True,True,True,False,False,False
498526,False,False,True,True,True,True,True,False,False,False


In [30]:
data.isna()

,level_0,date,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code,Time,Date
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
498523,False,False,True,True,True,True,True,False,False,False
498524,False,False,True,True,True,True,True,False,False,False
498525,False,False,True,True,True,True,True,False,False,False
498526,False,False,True,True,True,True,True,False,False,False


In [31]:
data.info

<bound method DataFrame.info of         level_0                      date  temperature_2m  surface_pressure  \
0        Zurich 2010-01-01 00:00:00+00:00           0.894        942.275085   
1        Zurich 2010-01-01 01:00:00+00:00           0.494        941.824402   
2        Zurich 2010-01-01 02:00:00+00:00          -0.006        941.640625   
3        Zurich 2010-01-01 03:00:00+00:00          -0.306        941.112305   
4        Zurich 2010-01-01 04:00:00+00:00          -0.556        940.593262   
...         ...                       ...             ...               ...   
498523  Bolzano 2024-03-20 19:00:00+00:00             NaN               NaN   
498524  Bolzano 2024-03-20 20:00:00+00:00             NaN               NaN   
498525  Bolzano 2024-03-20 21:00:00+00:00             NaN               NaN   
498526  Bolzano 2024-03-20 22:00:00+00:00             NaN               NaN   
498527  Bolzano 2024-03-20 23:00:00+00:00             NaN               NaN   

        wind_speed_